In [ ]:
import pandas as pd
import os
import re
import logging
# import sys
from tkinter import filedialog

# raw_file = open(os.path.normpath("C:/Users/rjaques/PycharmProjects/HMDvalidation/2145-04_Heathrow_SCANNER_2021Lane_1_CL.hmd")
# raw_file = open(os.path.normpath("G:/2019-20 SCANNER Data/WDM/Northern_Ireland/TNI RAV 2019- CL1.HMD"),
# raw_file = open(os.path.normpath("G:/2019-20 SCANNER Data/WDM/Wales/Torfaen/TORFAEN RAV 2019- CR1.HMD"),
# mode='r')

# raw_hmdif = raw_file.readlines()
# raw_file.close()

logger = logging.getLogger('HMDvalidation')
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

curr_dir = os.getcwd()

logger.info('Current directory' + curr_dir)

file_path_variable = filedialog.askdirectory(initialdir=curr_dir, title='Please select a directory containing the data')

if len(file_path_variable) > 0:
    logger.info('Working with ' + file_path_variable)
# return tempdir

# file_list = os.listdir(file_path_variable)
# file_list = glob.glob(file_path_variable)

file_list = list()

file_list += [each for each in os.listdir(file_path_variable) if each.endswith('.hmd')]

print(file_list)


# HMSTART check

# check for only one HMSTART

def line_counting(data_to_process, search_string):
    # Counts the number of lines in a list with the search string in it.

    count = 0

    for line in data_to_process:
        if search_string in line:
            count += 1

    return count


def record_testing(data_to_process, list_of_strings, correct_count):
    # find out if the number of items in a list containing the string is the same as the correct_count

    for field in list_of_strings:
        if line_counting(data_to_process, field) != 1:
            print(field + " is missing")

    return


def check_record_counts(start_string, end_string, file_to_check):
    start_index = [i for i, item in enumerate(file_to_check) if item.startswith(start_string)][0]
    end_index = [i for i, item in enumerate(file_to_check) if item.startswith(end_string)][0]

    length_between_indexes = len(file_to_check[start_index:end_index]) + 1

    res = re.findall(r'\d+', (list(filter(lambda x: end_string in x, raw_hmdif))[0]))

    # When the record count does not equal the value in the *END record shout...

    if int(res[0]) != int(length_between_indexes):
        print('The ' + end_string + ' record count is wrong')

    #    print('Record count OK')

    return


# for file in file_list:
#    print(file)
file = file_path_variable + "/" + file_list[0]

print(file)
# print(os.path.normpath(file[0]))
raw_file = open(file, "r")

raw_hmdif = raw_file.readlines()

raw_file.close()

list_of_records = ['HMSTART', 'TSTART', 'TEND', 'DSTART', 'DEND', 'HMEND']

record_testing(raw_hmdif, list_of_records, 1)

# check the record counts
# using filter() + lambda
# to get string with substring
# res = re.findall(r'\d+', (list(filter(lambda x: 'HMEND' in x, raw_hmdif))[0]))

check_record_counts('HMSTART', 'HMEND', raw_hmdif)
check_record_counts('TSTART', 'TEND', raw_hmdif)
check_record_counts('DSTART', 'DEND', raw_hmdif)

# check for only the values in the list_of_records

raw_hmdif = [w.replace(';', '') for w in raw_hmdif]
raw_hmdif = [w.replace('\n', '') for w in raw_hmdif]
# raw_hmdif = [w.replace('\\', '') for w in raw_hmdif]

# find the DSTART index

start_line = raw_hmdif.index('DSTART')

# all the real data is after DSTART so :

hmd_data = raw_hmdif[(start_line+1):]

In [ ]:
# print(len(hmd_data))

# remove the control HMEND and DEND records

hmd_data = hmd_data[:-2]

print(len(hmd_data))

list_out = list()
el: str = ""
survey: str = ""
section: str = ""
observation: str = ""

#hmdif_df = pd.DataFrame(hmd_data)

#print(len(hmdif_df))

# list_out = list()
# el: str
# survey: str
for el in hmd_data:
    if el.split('\\')[0] == 'SURVEY':
        survey = el.rsplit('\\', 1)[1]
    if el.split('\\')[0] == 'SECTION':
        section = el.rsplit('\\', 1)[1]
    if el.split('\\')[0] == 'OBSERV':
        observation = el.rsplit('\\', 1)[1]
    if el.split('\\')[0] == 'OBVAL':
        el = el.rsplit('\\', 1)[1]
        # print(f"{survey},{section},{observation},{el}")
        # list_out.append(survey,section,observation,el)
        comma: str = ","
        # seq = str()
        line = ['SURVEY',',', survey,',','SECTION',',', section,',','OBSERV',',',observation,',','OBVAL',',',el]
        seq = " ".join(line)
        # print(seq)
        list_out.append(seq)
        # print(len(list_out))

hmdif_df = pd.DataFrame(list_out)

In [ ]:
# split each record into dataframe columns based on the separating value being a ','
hmdif_df2 = hmdif_df[0].str.split(',', expand=True)